# Inclusion of scripts in SoS workflows

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * A Python function with a string as the first parameter can be written in a script style
  * Scripts are included in verbatim by default
  * Indentation is not required but recommended
  * `expand=True` turns the script into a Python f-string
  * `expand="l r"` can be used to specify an alternative delimiter for string interpolation

## SoS "actions"

SoS defines a number of **actions**, which are simply Python functions that follow a specific set of conventions. For example, an action `sh` is a python function that executes its first parameter as a shell script:

In [1]:
sh('echo Hello world')

Hello world


When you execute a SoS script from command line or SoS Notebook, these functions are automatically imported and can be used directly, and you can use them just like any other Python functions. For example, you can compose a script using Python string formatting:

In [2]:
greeting = "Hello world"
sh(f'echo {greeting}')

Hello world


Here we define a Python string `greeting` and use the `sh` action to execute a shell script. A [Python f-string](https://www.python.org/dev/peps/pep-0498/) is used to compose the script with the defined variable.

<div class="bs-callout bs-callout-primary" role="alert">
  <h4>Python f-string</h4>
  <p>SoS uses Python f-string extensively. Please read <a href="https://www.python.org/dev/peps/pep-0498/">PEP498</a> or one of the online tutorials on how to use Python f-strings if you are not familiar with it.</p>  
</div>

When the scripts get longer, you will have to use Python multi-line strings to pass them to the action. Here is an example for the use of a `R` action to execute a longer `R` script:

In [3]:
R("""\
x <- 5
if(x >= 0) {
    print("Non-negative number")
} else {
    print("Negative number")
}
""")

[1] "Non-negative number"


The use of format string in these cases become more complicated. First, you will need to use multi-line f-string (`f'''x'''` and `f"""x"""`). Second, when the script itself contains braces, they will need to be doubled to avoid being interpreted as Python expressions.

Consequently, a `R` script that uses a Python variable `my_num` needs to be written as follows:

In [4]:
my_num = -1

R(f"""\
x <- {my_num}
if(x >= 0) {{
    print("Non-negative number")
}} else {{
    print("Negative number")
}}
""")

[1] "Negative number"


## Script style function calls <a id="Script_style_function_call"></a>

The f-string in the last example is not quite readable, error-prone, and difficult to maintain, especially when the script contains multiple braces and variables. For this reason, SoS introduces a special syntax that allows you to write Python functions that accept a script (string) as the first parameter in a special script format.

For example,

In [5]:
R("""\
x <- 5
if(x >= 0) {
    print("Non-negative number")
} else {
    print("Negative number")
}
""")

[1] "Non-negative number"


can be written as

In [6]:
R:
x <- 5
if(x >= 0) {
    print("Non-negative number")
} else {
    print("Negative number")
}

[1] "Non-negative number"


or better as

In [7]:
R:
    x <- 5
    if(x >= 0) {
        print("Non-negative number")
    } else {
        print("Negative number")
    }

[1] "Non-negative number"


Indentation is optional and SoS will automatically dedent the included scripts.

<div class="bs-callout bs-callout-info" role="alert">
  <h4>Indentation of scripts</h4>
  <p>The indentation of scripts in the script style is optional but recommended because it makes the end scripts easier to read </p>
    <pre>
    R:
       cat('this is R')
    </pre>
    <pre>
    python:
       print('this is python')
    </pre>
</div>

## <a id="option-expand"></a> The `expand` option

When option `expand=True` is specified, the included script will be treated as a Python f-string. For example,

In [8]:
my_num = -1

R(f"""\
x <- {my_num}
if(x >= 0) {{
    print("Non-negative number")
}} else {{
    print("Negative number")
}}
""")

[1] "Negative number"


can be written as

In [9]:
my_num = -1

R: expand=True
    x <- {my_num}
    if(x >= 0) {{
        print("Non-negative number")
    }} else {{
        print("Negative number")
    }}

[1] "Negative number"


Note that SoS Notebook automatically highlights the interpolated parts of the included script, which makes it much easier to differentiate Python expressions from the original R script.

<div class="bs-callout bs-callout-info" role="alert">
  <h4>Use of alternative sigils</h4>
  <p>When the included scripts have braces, it is easier to use an alternative sigil for string interpolation.</p>
</div>

Because the included script has two pairs of braces, it is necessary to double them so that they are not treated as Python expressions. In these cases, it is actually easier to use a different set of sigil (delimiters) for string interpolation. This can be done using the `expand` option as follows:

In [10]:
my_num = -1

R: expand='${ }'
    x <- ${my_num}
    if(x >= 0) {
        print("Non-negative number")
    } else {
        print("Negative number")
    }

[1] "Negative number"


The sigil should be specified as a string with left and right sigil separated by a space. You can use any pair of sigils as long as they do not cause confusion. 

## Indented script format

What you have seen so far are function calls without Python control structure. Because SoS allows you to execute the same statements repeatedly for different inputs ([substeps](input_substeps.html) and stop the execution of substeps through actions such as [`skip_if` and `done_if`](control_actions.html), there is rarely a need to call SoS actions within the `if...else...` block or `for` or `while` loops.

However, if you do need to call SoS actions inside a Python control structure, you can still use the script format for it. For example, instead of writing

In [11]:
for i in range(3):
    sh(f'echo {i}')

0


1


2


You can write the `sh` action in the format of

In [12]:
for i in range(3):
    sh: expand=True
        echo {i}

0


1


2


Indentation for the included script is optional

In [13]:
for i in range(3):
    sh: expand=True
echo {i}

0


1


2


but is required in more complex cases to avoid ambiguity. For example, 

In [14]:
%run --var 2
parameter: var=int

if var > 1:
    python:
        print('Python script for var > 1')
else:
    python:
        print('Python script for var <= 1')
    

Python script for var > 1


cannot be written as

```
if var > 1:
    python:
print('Python script for var > 1')
else:
    python:
print('Python script for var <= 1')
```

because the `else:` block would be considered as part of the first `python` script.

## Further reading
* [SoS Actions and common action options](sos_actions.html)
* [substeps of SoS](input_substeps.html)
* [Actions for controlling execution of substeps](control_actions.html)